In [1]:
import ROOT as r

Welcome to JupyROOT 6.24/06


In [2]:
h_baseline_pt=r.TH1F("baseline_pt","baseline_pt",100,0,100)

h_istruth_it_ii_pt=r.TH1F("istruth_istight_isiso_pt","istight_isiso",100,0,100)
h_istruth_it_ni_pt=r.TH1F("istruth_istight_noiso_pt","istight_noiso",100,0,100)
h_istruth_nt_ii_pt=r.TH1F("istruth_notight_isiso_pt","notight_isiso",100,0,100)
h_istruth_nt_ni_pt=r.TH1F("istruth_notight_noiso_pt","notight_noiso",100,0,100)

h_notruth_it_ii_pt=r.TH1F("notruth_istight_isiso_pt","istight_isiso",100,0,100)
h_notruth_it_ni_pt=r.TH1F("notruth_istight_noiso_pt","istight_noiso",100,0,100)
h_notruth_nt_ii_pt=r.TH1F("notruth_notight_isiso_pt","notight_isiso",100,0,100)
h_notruth_nt_ni_pt=r.TH1F("notruth_notight_noiso_pt","notight_noiso",100,0,100)

f=r.TFile("/data/maclwong/Ben_Bkg_Samples/v2/user.bhodkins.700401.Znunugamma.mc20e.v2.0_ANALYSIS.root/user.bhodkins.42165174._000001.ANALYSIS.root","RO")
t=f.Get("analysis")

eventcount=0
totalevents=t.GetEntriesFast()
for e in t:

    eventcount+=1
    if eventcount%50000 == 0:
        print(f"Processed {eventcount:6d}/{totalevents} events")
    #if eventcount>10000:
    #    break
    
    #=================================================================
    # preselection
    #
    met=e.met_met_NOSYS
    met_phi=e.met_phi_NOSYS
    met_tlv=r.TLorentzVector()
    met_tlv.SetPtEtaPhiM(met,0,met_phi,0)

    # MET requirement, for now just look at low MET
    if e.met_met_NOSYS>250000: continue

    # veto events with leptons
    nelectrons = len(e.el_pt_NOSYS)
    nmuons     = len(e.mu_pt_NOSYS)
    if (nelectrons+nmuons)>0: continue

    # no b-tagged jets, at least one jet with pT>100 GeV, no jets near MET
    jets=[]
    mindphijetsmet=999
    btags=0
    for j in range(len(e.jet_pt_NOSYS)):
        tlv=r.TLorentzVector()
        tlv.SetPtEtaPhiM(e.jet_pt_NOSYS[j],
                         e.jet_eta[j],
                         e.jet_phi[j],
                         0)
        jets.append(tlv)
        mindphijetsmet=min(mindphijetsmet,tlv.DeltaPhi(met_tlv))
        if ord(e.jet_btag_select[j])!=0: btags+=1
    if jets[0].Pt()<100000: continue
    if mindphijetsmet<0.4: continue
    if btags>0: continue

    # at least one loose photon with pT>10 GeV, no photons near jets
    photons={}
    leading_baseline_photon_index=-1
    for p in range(len(e.ph_pt_NOSYS)):
        if not e.ph_select_baseline_NOSYS[p]: 
            continue

        tlv=r.TLorentzVector()
        tlv.SetPtEtaPhiM(e.ph_pt_NOSYS[p],
                         e.ph_eta[p],
                         e.ph_phi[p],
                         0)

        if tlv.Pt()<10000:
            continue
        
        if abs(tlv.Eta())>2.37 or (abs(tlv.Eta())>1.37 and abs(tlv.Eta())<1.52):
            continue
            
        mindrphjet=999
        for j in jets:
            mindrphjet=min(mindrphjet,tlv.DeltaPhi(j))
        if mindrphjet<0.4: continue
    
        photons[p]=tlv
        if leading_baseline_photon_index<0:
            leading_baseline_photon_index=p


    if len(photons)<1: continue

    h_baseline_pt.Fill(e.ph_pt_NOSYS[leading_baseline_photon_index]/1000.)
    
    ## overlap removal for vgamma samples, hold off for now
    # if not e.in_vgamma_overlap_7: continue
    #=================================================================

    #=================================================================
    # now find the photons that we want to use
    #
    photon_index=-1
    for p in photons:
        tlv=photons[p]
        #print(f"0x{e.ph_isEM_NOSYS[p]:08x} & 0x45fc01 = 0x{e.ph_isEM_NOSYS[p] & 0x45fc01:08x}, Tight={ord(e.ph_select_tightID_NOSYS[p])}")
        if (e.ph_isEM_NOSYS[p] & 0x45fc01) != 0:
            continue
        photon_index=p
        break
    if photon_index<0:
        continue
    #=================================================================

    #=================================================================
    # now we have the index of the photon to keep.  Now figure out if this is a truth-matched photon or not.
    # will need to fix this whenever Giordon et al decide on a truth matching scheme.
    #print(f"{e.ph_truthType[photon_index]} {e.ph_truthOrigin[photon_index]} {e.ph_truthpdgId[photon_index]}")
    ph_truthmatch=e.ph_truthType[photon_index]!=16 and e.ph_truthType[photon_index]!=0

    # the tight and isolated flags are easier:
    ph_tight=ord(e.ph_select_tightID_NOSYS [photon_index])
    ph_iso  =ord(e.ph_select_tightIso_NOSYS[photon_index])

    if ph_truthmatch:
        if       ph_tight and     ph_iso: h_istruth_it_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif     ph_tight and not ph_iso: h_istruth_it_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif not ph_tight and     ph_iso: h_istruth_nt_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif not ph_tight and not ph_iso: h_istruth_nt_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
    else:
        if       ph_tight and     ph_iso: h_notruth_it_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif     ph_tight and not ph_iso: h_notruth_it_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif not ph_tight and     ph_iso: h_notruth_nt_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif not ph_tight and not ph_iso: h_notruth_nt_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
    #=================================================================

    
print(f"                 baseline: {h_baseline_pt.GetSumOfWeights()}")

print(f" truth,  tight,  isolated: {h_istruth_it_ii_pt.GetSumOfWeights()}")
print(f" truth,  tight, !isolated: {h_istruth_it_ni_pt.GetSumOfWeights()}")
print(f" truth, !tight,  isolated: {h_istruth_nt_ii_pt.GetSumOfWeights()}")
print(f" truth, !tight,  isolated: {h_istruth_nt_ni_pt.GetSumOfWeights()}")

print(f"!truth,  tight,  isolated: {h_notruth_it_ii_pt.GetSumOfWeights()}")
print(f"!truth,  tight, !isolated: {h_notruth_it_ni_pt.GetSumOfWeights()}")
print(f"!truth, !tight,  isolated: {h_notruth_nt_ii_pt.GetSumOfWeights()}")
print(f"!truth, !tight,  isolated: {h_notruth_nt_ni_pt.GetSumOfWeights()}")

Processed  50000/872532 events
Processed 100000/872532 events
Processed 150000/872532 events
Processed 200000/872532 events
Processed 250000/872532 events
Processed 300000/872532 events
Processed 350000/872532 events
Processed 400000/872532 events
Processed 450000/872532 events
Processed 500000/872532 events
Processed 550000/872532 events
Processed 600000/872532 events
Processed 650000/872532 events
Processed 700000/872532 events
Processed 750000/872532 events
Processed 800000/872532 events
Processed 850000/872532 events
                 baseline: 956.0
 truth,  tight,  isolated: 544.0
 truth,  tight, !isolated: 99.0
 truth, !tight,  isolated: 80.0
 truth, !tight,  isolated: 24.0
!truth,  tight,  isolated: 23.0
!truth,  tight, !isolated: 14.0
!truth, !tight,  isolated: 15.0
!truth, !tight,  isolated: 11.0


Error in <TList::Clear>: A list is accessing an object (0x8539970) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x85910e0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x8591510) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x859c3e0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x859c750) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x859cbc0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x859d060) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x859d430) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x859d900) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x859ec60) alread